# Create queries for simple flask api with pandas

Goal is to replace the call to the Firestore Database with a fetch from local disk.

I.e. we will save a csv file on the app engine machine.

In [ ]:
import pandas as pd

In [ ]:
data_dir = '../../data/wikivoyage/processed/'

data_file  = data_dir + 'wikivoyage_destinations.csv'

### Read data

In [ ]:
df = pd.read_csv(data_file)

## Build queries

### based on geo

ne_lat, ne_lng, sw_lat, and sw_lng will be given:

`&ne_lat=43.97363914475397&ne_lng=5.173845810128569&sw_lat=38.69043481932856&sw_lng=-0.5720037992464313`

In [ ]:
ne_lat, ne_lng, sw_lat, sw_lng = 43.9, 5.17, 38.7, -0.57

Apply filter:

In [ ]:
(
    df
    .loc[lambda df: (df['lat'] >= sw_lat) & (df['lat'] < ne_lat)]
    .loc[lambda df: (df['lng'] >= sw_lng) & (df['lng'] < ne_lng)]
).head()

### sorting based on OSP importance

In order to get some randomness

**TODO**: not yet built in into new dataset, so sample random for now.

In [ ]:
(
    df
    .sample(6)
#     .where('osp_importance', '<=', normal(osp_importance_mu, osp_importance_std))
#     .order_by('osp_importance', direction=firestore.Query.DESCENDING)
)

### Rename for front-end

make sure feature names are the same as used in React

In [ ]:
(
    df
    .rename(columns={'pageid': 'id', 'title':'name', 'country':'country_name', 'articletype':'dest_wiki_type', 
                    'lat':'latitude', 'lng':'longitude'})
).head()

### Convert to json

In [ ]:
df.sample(2).to_dict(orient='records')

Done.